In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Данные:
Сопротивление проводника = удельное сопровтивление * длина / площадь

In [2]:
# Создаём сэмпл
n_samples = 500

resistivity = np.random.choice(15, n_samples) + 2 # mOhm * mm2 / m - удельное сопротивление 
length = np.random.choice(120, n_samples) + 1 # m
width = np.random.choice(5, n_samples) + 1 # mm
height = np.random.choice(5, n_samples) + 1 # mm
density = np.random.choice(20, n_samples) + 4 # г/ см3 - плотность

resistance =  resistivity * length / (width * height) # mOhm - сопротивление проводника

data = pd.DataFrame(
    {'resistivity': resistivity, 
     'length': length, 'width': width, 'height': height,
     'density': density,
     'resistance': resistance
    })
data.head(5) 

,resistivity,length,width,height,density,resistance
0,11,114,4,5,20,62.70
1,2,108,4,2,12,27.00
2,4,49,5,5,20,7.84
3,7,84,2,5,15,58.80
4,8,109,2,4,9,109.00


Функция построения линейной модели

In [3]:
from sklearn.metrics import mean_absolute_error

def linear_reg(X, y):
    reg = LinearRegression().fit(X, y)
    print('Weights: {}'.format(reg.coef_))
    print('Bias: {}'.format(reg.intercept_))

    pred_values = reg.predict(X)
    print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Модель по всевм данным

In [4]:
X = data[['resistivity', 'length', 'width', 'height', 'density']]
y = data['resistance']
linear_reg(X, y)

Weights: [ 14.58343181   2.14680138 -54.10551944 -53.36554035   0.29940018]
Bias: 182.3563904225541
Error: 81.78198414213222


Модель  по данным без лишнего параметра - плотности

In [5]:
X = data[['resistivity', 'length', 'width', 'height']]
y = data['resistance']
linear_reg(X, y)

Weights: [ 14.60391055   2.14393927 -54.124548   -53.30486971]
Bias: 186.30981862317032
Error: 81.82233748916467


Новый признак - площадь сечения проводника

In [6]:
data['area'] = data['height'] * data['width']
data.head(5)

,resistivity,length,width,height,density,resistance,area
0,11,114,4,5,20,62.70,20
1,2,108,4,2,12,27.00,8
2,4,49,5,5,20,7.84,25
3,7,84,2,5,15,58.80,10
4,8,109,2,4,9,109.00,8


In [7]:
X = data[['resistivity', 'length', 'area']]
y = data['resistance']
linear_reg(X, y)

Weights: [ 14.94658739   2.18093003 -14.20305009]
Bias: -12.920928767758255
Error: 87.40219220874447


Смещение (-15), т.е. при нулевой длине сопротивление "будет" отрицательным вместо нулевого! 

Новый признак - отношение длины и площади

In [8]:
data['ratio'] = data['length'] / data['area']
data.head(5)

,resistivity,length,width,height,density,resistance,area,ratio
0,11,114,4,5,20,62.70,20,5.700
1,2,108,4,2,12,27.00,8,13.500
2,4,49,5,5,20,7.84,25,1.960
3,7,84,2,5,15,58.80,10,8.400
4,8,109,2,4,9,109.00,8,13.625


In [9]:
X = data[['resistivity', 'ratio']]
y = data['resistance']
linear_reg(X, y)

Weights: [13.07444291  9.88533387]
Bias: -120.1259216045537
Error: 40.58064202920765


Хоть и ошибка в два раза меньше, модель не корректна

Новый признак - произведение удельного сопротивления и отношения длины и площади

In [10]:
data['mult'] = data['resistivity'] * data['ratio']
data.head(5)

,resistivity,length,width,height,density,resistance,area,ratio,mult
0,11,114,4,5,20,62.70,20,5.700,62.70
1,2,108,4,2,12,27.00,8,13.500,27.00
2,4,49,5,5,20,7.84,25,1.960,7.84
3,7,84,2,5,15,58.80,10,8.400,58.80
4,8,109,2,4,9,109.00,8,13.625,109.00


In [11]:
X = data[['mult']]
y = data['resistance']
linear_reg(X, y)

Weights: [1.]
Bias: 2.842170943040401e-14
Error: 2.4863111569572993e-14


Ошибка 10Е-14! Практически идеально. Хорошо когда знаешь ответ! А вот в реальности - ...